<a href="https://colab.research.google.com/github/cwolffff/m00sic/blob/main/notebooks/musicgen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prelims

### Install dependencies

In [3]:
#@test {"output": "ignore"}

# This only needs to be run ONCE:
# !pip install -qU magenta pyfluidsynth pretty_midi

### Import packages

In [4]:
# Hack to allow python to pick up the fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
    if lib == "fluidsynth":
        return "libfluidsynth.so.1"
    else:
        return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

import magenta
import note_seq
from note_seq.protobuf import music_pb2

print(f"Magenta version: {magenta.__version__}")

Magenta version: 2.1.3


### Test run

In [ ]:
twinkle_twinkle = music_pb2.NoteSequence()

# Add the notes to the sequence.
twinkle_twinkle.notes.add(pitch=60, start_time=0.0, end_time=0.5, velocity=80)
twinkle_twinkle.notes.add(pitch=60, start_time=0.5, end_time=1.0, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=1.0, end_time=1.5, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=1.5, end_time=2.0, velocity=80)
twinkle_twinkle.notes.add(pitch=69, start_time=2.0, end_time=2.5, velocity=80)
twinkle_twinkle.notes.add(pitch=69, start_time=2.5, end_time=3.0, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=3.0, end_time=4.0, velocity=80)
twinkle_twinkle.notes.add(pitch=65, start_time=4.0, end_time=4.5, velocity=80)
twinkle_twinkle.notes.add(pitch=65, start_time=4.5, end_time=5.0, velocity=80)
twinkle_twinkle.notes.add(pitch=64, start_time=5.0, end_time=5.5, velocity=80)
twinkle_twinkle.notes.add(pitch=64, start_time=5.5, end_time=6.0, velocity=80)
twinkle_twinkle.notes.add(pitch=62, start_time=6.0, end_time=6.5, velocity=80)
twinkle_twinkle.notes.add(pitch=62, start_time=6.5, end_time=7.0, velocity=80)
twinkle_twinkle.notes.add(pitch=60, start_time=7.0, end_time=8.0, velocity=80) 
twinkle_twinkle.total_time = 8

twinkle_twinkle.tempos.add(qpm=60);

# This is a colab utility method that visualizes a NoteSequence.
note_seq.plot_sequence(twinkle_twinkle)

# This is a colab utility method that plays a NoteSequence.
note_seq.play_sequence(twinkle_twinkle,synth=note_seq.fluidsynth)

## Step 1: Opening chords

In [ ]:
import random

notes_per_key = {
    "CM": [21, 23, 24, 26, 28, 29, 31, 33, 35, 36, 38, 40, 41, 43, 45, 47, 48, 50, 52, 53, 55, 57, 59, 60, 62, 64, 65, 67, 69, 71, 72, 74, 76, 77, 79, 81, 83, 84, 86, 88, 89, 91, 93, 95, 96, 98, 100, 101, 103, 105, 107, 108],
    "GM": [21,23,24,26,28,30,31,33,35,36,38,40,42,43,45,47,48,50,52,54,55,57,59,60,62,64,66,67,69,71,72,74,76,78,79,81,83,84,86,88,90,91,93,95,96,98,100,102,103,105,107,108],
    "DM": [2,3,4,5],
    "AM": [2,3,4,5],
    "EM": [2,3,4,5],
    "BM": [2,3,4,5],
    "FSM": [2,3,4,5],
    "CSM": [2,3,4,5],
    "Am": [2,3,4,5],
    "Em": [2,3,4,5],
    "Bm": [2,3,4,5],
    "FSm": [2,3,4,5],
    "CSm": [2,3,4,5],
    "GSm": [2,3,4,5],
    "DSm": [2,3,4,5],
    "ASm": [2,3,4,5]
}

tonic_note_per_key = {
    "CM": 60,
    "GM": 67,
    "DM": 11,
    "AM": 11,
    "EM": 11,
    "BM": 11,
    "FSM": 11,
    "CSM": 11,
    "Am": 11,
    "Em": 11,
    "Bm": 11,
    "FSm": 11,
    "CSm": 11,
    "GSm": 11,
    "DSm": 11,
    "ASm": 11
}

chord_steps = {
    1: [0,4,7]
}

def pick_me_a_key_yo():
  return random.choice(list(notes_per_key.keys()))


## Step 2: Chord progression

In [ ]:
a_key = pick_me_a_key_yo()

# returns tonic + some offset
def get_me_a_starting_note_yo(key, offset):
  if offset == "I":
    return tonic_note_per_key[key]
  elif offset == "II":
    return notes_per_key[key][notes_per_key[key].index(tonic_note_per_key[key])+1]
  elif offset == "III":
    return notes_per_key[key][notes_per_key[key].index(tonic_note_per_key[key])+2]
  elif offset == "IV":
    return notes_per_key[key][notes_per_key[key].index(tonic_note_per_key[key])+3]
  elif offset == "V":
    return notes_per_key[key][notes_per_key[key].index(tonic_note_per_key[key])+4]
  elif offset == "VI":
    return notes_per_key[key][notes_per_key[key].index(tonic_note_per_key[key])+5]
  else:
    print("Invalid offset")
    return None

def get_chord(key, note, chord):
  if key not in notes_per_key:
    print("Invalid Key")
    return None

  # note must be valid in key
  if note not in notes_per_key[key]:
    print("Invalid Note")
    return None

  if chord not in chord_steps:
    print("Invalid Chord")
    return None

  return [note+i for i in chord_steps[chord]]

def get_chord_first_inversion(key, note, chord):
  notes = get_chord(key,note,chord)
  return notes[1:] + notes[:1]
  
def get_chord_second_inversion(key, note, chord):
  notes = get_chord(key,note,chord)
  return notes[-1:] + notes[:-1]

def build_chord_progression(key):

  note1 = get_me_a_starting_note_yo(key, "I")
  chord1 = get_chord(key, note1, 1)

  note2 = get_me_a_starting_note_yo(key, "V")
  chord2 = get_chord_second_inversion(key, note2, 1)

  note3 = get_me_a_starting_note_yo(key, "VI")
  chord3 = get_chord_first_inversion(key, note3, 1)

  note4 = get_me_a_starting_note_yo(key, "IV")
  chord4 = get_chord_second_inversion(key, note4, 1)

  return [chord1, chord2, chord3, chord4]


print(get_chord("CM", 60, 1))
print(get_chord_first_inversion("CM", 60, 1))
print(get_chord_second_inversion("CM", 60, 1))

print(get_me_a_starting_note_yo("CM", "I"))
print(get_me_a_starting_note_yo("CM", "II"))
print(get_me_a_starting_note_yo("CM", "III"))
print(get_me_a_starting_note_yo("CM", "IV"))
print(get_me_a_starting_note_yo("CM", "V"))

print(build_chord_progression("CM"))

from note_seq.protobuf import music_pb2

twinkle_twinkle = music_pb2.NoteSequence()

# Add the notes to the sequence.
twinkle_twinkle.notes.add(pitch=60, start_time=0.0, end_time=1.0, velocity=80)
twinkle_twinkle.notes.add(pitch=64, start_time=0.0, end_time=1.0, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=0.0, end_time=1.0, velocity=80)
twinkle_twinkle.notes.add(pitch=74, start_time=1.0, end_time=2.0, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=1.0, end_time=2.0, velocity=80)
twinkle_twinkle.notes.add(pitch=71, start_time=1.0, end_time=2.0, velocity=80)
twinkle_twinkle.notes.add(pitch=73, start_time=2.0, end_time=3.0, velocity=80)
twinkle_twinkle.notes.add(pitch=76, start_time=2.0, end_time=4.0, velocity=80)
twinkle_twinkle.notes.add(pitch=69, start_time=0, end_time=3.0, velocity=80)
twinkle_twinkle.notes.add(pitch=69, start_time=2.0, end_time=3.0, velocity=80)
twinkle_twinkle.notes.add(pitch=72, start_time=3.0, end_time=4.0, velocity=80)
twinkle_twinkle.notes.add(pitch=65, start_time=33.0, end_time=4.0, velocity=80)
twinkle_twinkle.total_time = 4

twinkle_twinkle.tempos.add(qpm=60);

# This is a colab utility method that visualizes a NoteSequence.
note_seq.plot_sequence(twinkle_twinkle)

# This is a colab utility method that plays a NoteSequence.
note_seq.play_sequence(twinkle_twinkle,synth=note_seq.fluidsynth)

## Step 3: Melody

## Step 4: Dynamics

## Step 5: Structure